### Problem set 7
#### Submitted by: Abu Huraira

Consider the following declarations:

In [43]:

type Name = String
data Grade = A | B | C | D
data Student = Student Name Grade
data Vec2 = Vec2 Int Int
data Result a
    = Success a
    | Failure String
    deriving Show

dup f x = f x x
dip f x = f (f x x)
twice f x = f (f x)

1. Specify the (most generic) types of dup, dip, and twice. Infer the type for each of the following expressions or specify a type error. Justify your answer by providing a step-by-step reasoning process. Assume that the type of integer literals is Int:

**Solution:**

The most generic types of `dup`, `dip`, and `twice` are as follows:

```haskell
dup :: (a -> a -> b) -> a -> b

dip :: (a -> a -> a) -> a -> a -> a

twice :: (a -> a) -> a -> a
```

(a) `dup Vec2`
    
= `Int -> Vec2`

(b) `dup (dip (*)) 5`

= `Num b => b`
    
(c) `twice dip`

= `(t -> t -> t) -> t -> t -> t`

(d) `dip dip`

= TypeError couldn't match type `t` with `t -> t -> t`

(e) `twice twice twice`

= (t -> t) -> t -> t

2. Using explicit recursion, implement function `studentsWithC :: [Student] -> [Name]` that returns a list of names of students with C grade:

```haskell
studensWithA [Student "Jack" C, Student "Jane" A]
-- ["Jack"]
```
Note: you cannot use (==) to compare grades for equality.

**Solution:**

In [44]:
studentsWithC :: [Student] -> [Name]
studentsWithC [] = []
studentsWithC ((Student name grade):xs) = case grade of
    C -> name : studentsWithC xs
    _ -> studentsWithC xs

studentsWithC [Student "Jack" C, Student "Jane" A]

["Jack"]

3. (a) Implement a polymorphic higher-order function `whileSuccess :: (a -> Result a) -> a -> a` that applies a function repeatedly, as long as the result is a Success, otherwise, it returns the last value.

```haskell
    f n | n > 100 = Failure "input is too large"
        | otherwise = Success (2 * n)
    example1 = whileSuccess f 1 -- 64
```

In [49]:
whileSuccess :: (a -> Result a) -> a -> a
whileSuccess f x =
    case f x of
        Success newX -> whileSuccess f newX
        Failure _    -> x

f n | n > 100 = Failure "input is too large"
    | otherwise = Success (2 * n)

example1 = whileSuccess f 1
print example1

128

(b) Implement a polymorphic higher-order function
```haskell
    applyResult :: Result (a -> b) -> Result a -> Result b
```
that applies a given function to a given argument when both are available, and returns the first (leftmost) error message otherwise.
```haskell
    applyResult (Success length) (Success [1, 2, 3]) -- Success 3
    applyResult (Failure "no function") (Failure "no arg") -- Failure "no function"
```

In [46]:
applyResult :: Result (a -> b) -> Result a -> Result b
applyResult (Success f) (Success x) = Success (f x)
applyResult (Failure err1) _ = Failure err1
applyResult _ (Failure err2) = Failure err2

print (applyResult (Success length) (Success [1, 2, 3]))
applyResult (Failure "no function") (Failure "no arg")

Success 3

Failure "no function"

(c) Implement a polymorphic higher-order function
```haskell
    fromResult :: (a -> b) -> (String -> b) -> Result a -> b
```
that processes any given result:
```haskell
    fromResult (+1) length (Success 3) -- 4
    fromResult (+1) length (Failure "not a number") -- 12
```


In [47]:
fromResult :: (a -> b) -> (String -> b) -> Result a -> b
fromResult onSuccess onFailure result =
    case result of
        Success x -> onSuccess x
        Failure err -> onFailure err

fromResult (+1) length (Success 3)
fromResult (+1) length (Failure "not a number")

4

12

(d) Implement a polymorphic higher-order function
```haskell
    combineResultsWith :: (a -> b -> c) -> Result a -> Result b -> Result c
```
that applies a function to two results, if both are present, and returns the first (leftmost)
error message otherwise.

```haskell
    combineResultsWith (+) (Result 2) (Result 3)
    -- Success 5
    combineResultsWith (+) (Failure "x is undefined") (Failure "crash")
    -- Failure "x is undefined"
```

In [48]:
combineResultsWith :: (a -> b -> c) -> Result a -> Result b -> Result c
combineResultsWith f (Success x) (Success y) = Success (f x y)
combineResultsWith _ (Failure err1) _ = Failure err1
combineResultsWith _ _ (Failure err2) = Failure err2

combineResultsWith (+) (Success 2) (Success 3)
combineResultsWith (+) (Failure "x is undefined") (Failure "crash")


Success 5

Failure "x is undefined"